In [27]:
from typing import Dict, List, TypedDict, Optional, Any
from langgraph.graph import StateGraph, END
import json
from openai import AzureOpenAI
import re
import logging

# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class MarketingState(TypedDict, total=False):
    messages: List[Dict[str, str]]
    conversation_count: int
    marketing_data: Dict[str, Any]
    max_conversation_turns: int

client = AzureOpenAI(
    azure_endpoint = "https://lmgdllaoidev2.openai.azure.com/",
    api_key = "b02e94fb7b024b63a18a0161de281416",
    api_version = "2025-01-01-preview"
)

def initialize_state(state: Dict) -> Dict:
    if "messages" not in state:
        state["messages"] = []
    if "conversation_count" not in state:
        state["conversation_count"] = 0
    if "marketing_data" not in state:
        state["marketing_data"] = {}
    if "max_conversation_turns" not in state:
        state["max_conversation_turns"] = 3
    return state

def get_missing_required_fields(marketing_data: Dict) -> List[str]:
    required_fields = ["customer_segment", "marketer_campaign_theme", "language", "preferred_channel"]
    return [field for field in required_fields if not marketing_data.get(field)]

def validate_marketing_data(marketing_data: Dict) -> Dict:
    common_events = ["ramadan", "eid", "christmas", "new year", "black friday", "cyber monday", 
        "valentine's day", "halloween", "back to school", "thanksgiving", 
        "diwali", "lunar new year", "hanukkah", "pride", "easter"]

    common_themes = ["new arrivals", "clearance sale", "seasonal collection", "loyalty program", 
        "product launch", "reactivation", "welcome back", "anniversary", "flash sale", 
        "grand opening", "membership", "renewal", "upgrade", "referral", "testimonial",
        "white wednesday", "sale", "generic", "promotional"]

    # Only move themes if they clearly match a theme — don't assume defaults
    if (marketing_data.get("ongoing_event") and 
        any(theme in marketing_data["ongoing_event"].lower() for theme in common_themes)):
        if not marketing_data.get("marketer_campaign_theme"):
            marketing_data["marketer_campaign_theme"] = marketing_data["ongoing_event"]
            marketing_data["ongoing_event"] = None

    # Don't force seasonal fallback — let the assistant ask for it
    return marketing_data

def extract_json_from_text(text: str) -> Dict:
    json_match = re.search(r'\{[\s\S]*\}', text, re.DOTALL)
    if json_match:
        try:
            return json.loads(json_match.group(0))
        except json.JSONDecodeError:
            pass

    code_block_match = re.search(r'```(?:json)?\s*([\s\S]*?)\s*```', text, re.DOTALL)
    if code_block_match:
        try:
            return json.loads(code_block_match.group(1))
        except json.JSONDecodeError:
            pass

    logger.warning("Could not extract valid JSON from text")
    return {}

def marketer_assistant(state: Dict) -> Dict:
    state = initialize_state(state)

    if len(state["messages"]) == 0:
        state["messages"].append({"role": "assistant", "content": "Hi, how can I assist you today?"})
        return state

    system_prompt = """
    You are a marketing assistant specialized in extracting campaign information from user conversations.
    Your goal is to extract the following:
    - customer_segment (e.g., Active Customers, New Users, etc.)
    - marketer_campaign_theme (e.g., New Arrivals, Sale, etc.)
    - language (English, Arabic, etc.)
    - preferred_channel (email, sms, whatsapp, etc.)
    Optional fields:
    - available_discount
    - coupon_code
    - ongoing_event

    Return a valid JSON object with these fields only.
    Do not assume values if not present.
    Do not include any explanatory text.
    """

    formatted_messages = [{"role": "system", "content": system_prompt}] + state["messages"]

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=formatted_messages,
            temperature=0.1,
        )
        response_content = response.choices[0].message.content
        marketing_data = extract_json_from_text(response_content)

        if not marketing_data:
            json_extraction_messages = formatted_messages + [
                {"role": "assistant", "content": response_content},
                {"role": "user", "content": "Please format your response as a valid JSON object only."}
            ]
            json_response = client.chat.completions.create(
                model="gpt-4",
                messages=json_extraction_messages,
                temperature=0.1,
                response_format={"type": "json_object"}
            )
            json_content = json_response.choices[0].message.content
            marketing_data = json.loads(json_content)

        marketing_data = validate_marketing_data(marketing_data)
        state["marketing_data"].update(marketing_data)

        missing_fields = get_missing_required_fields(state["marketing_data"])

        if missing_fields:
            next_question_map = {
                "customer_segment": "Could you please specify the customer segment you are targeting?",
                "marketer_campaign_theme": "What is the main theme or concept of this campaign?",
                "language": "Which language would you like the campaign message to be in?",
                "preferred_channel": "What is your preferred channel for this campaign? (e.g., Email, SMS, WhatsApp)"
            }

            # Special nudge: if theme is missing but event is known
            if "marketer_campaign_theme" in missing_fields and state["marketing_data"].get("ongoing_event"):
                event = state["marketing_data"]["ongoing_event"]
                state["messages"].append({
                    "role": "assistant",
                    "content": f"Since this is a {event} campaign, would you like to set a theme such as 'Seasonal Offer', 'Special Sale', or something else?"
                })
            else:
                next_field = missing_fields[0]
                question = next_question_map.get(next_field, f"Could you provide the {next_field}?")
                state["messages"].append({"role": "assistant", "content": question})
        else:
            state["messages"].append({
                "role": "assistant",
                "content": f"Thank you for providing all the necessary information. Here's the campaign data I've extracted:\n\n```json\n{json.dumps(state['marketing_data'], indent=2)}\n```"
            })

    except Exception as e:
        logger.error(f"Error extracting marketing data: {str(e)}")
        state["messages"].append({"role": "assistant", "content": f"Error: {str(e)}"})

    return state

def user_input(state: Dict, user_input_text: str) -> Dict:
    state = initialize_state(state)
    state["messages"].append({"role": "user", "content": user_input_text})
    state["conversation_count"] += 1
    return state

def should_end(state: Dict) -> str:
    state = initialize_state(state)
    all_fields_present = len(get_missing_required_fields(state.get("marketing_data", {}))) == 0
    max_turns_exceeded = state["conversation_count"] >= state["max_conversation_turns"]
    if all_fields_present or max_turns_exceeded:
        return "end"
    return "user_input"

def build_marketing_conversation_graph() -> StateGraph:
    builder = StateGraph(MarketingState)
    builder.add_node("marketer_assistant", marketer_assistant)
    builder.add_node("user_input", user_input)
    builder.add_conditional_edges("marketer_assistant", should_end, {"user_input": "user_input", "end": END})
    builder.add_edge("user_input", "marketer_assistant")
    builder.set_entry_point("marketer_assistant")
    return builder.compile()

def get_latest_assistant_message(state: Dict) -> str:
    if not state.get("messages"):
        return ""
    assistant_messages = [m for m in state["messages"] if m["role"] == "assistant"]
    return assistant_messages[-1]["content"] if assistant_messages else ""

def run_marketing_conversation(initial_input: str, max_turns: int = 5) -> Dict:
    state = initialize_state({"max_conversation_turns": max_turns})
    original_log_level = logging.getLogger().level
    logging.getLogger().setLevel(logging.ERROR)

    try:
        state = marketer_assistant(state)
        print(f"Assistant: {get_latest_assistant_message(state)}\n")

        print(f"User: {initial_input}")
        state = user_input(state, initial_input)

        state = marketer_assistant(state)
        print(f"Assistant: {get_latest_assistant_message(state)}\n")

        while len(get_missing_required_fields(state.get("marketing_data", {}))) > 0 and state["conversation_count"] < max_turns:
            user_message = input("User: ")
            state = user_input(state, user_message)
            state = marketer_assistant(state)
            print(f"Assistant: {get_latest_assistant_message(state)}\n")

    finally:
        logging.getLogger().setLevel(original_log_level)

    return state.get("marketing_data", {})

if __name__ == "__main__":
    initial_message = """I need to set up a campaign for Ramadan"""
    marketing_data = run_marketing_conversation(initial_message)
    print("\nFinal Marketing Data:")
    print(json.dumps(marketing_data, indent=2))

Assistant: Hi, how can I assist you today?

User: I need to set up a campaign for Ramadan
Assistant: Since this is a Ramadan campaign, would you like to set a theme such as 'Seasonal Offer', 'Special Sale', or something else?

User: new arrivals,flat_50,sms
Assistant: Thank you for providing all the necessary information. Here's the campaign data I've extracted:

```json
{
  "ongoing_event": "Ramadan",
  "customer_segment": "Ramadan",
  "marketer_campaign_theme": "New Arrivals",
  "language": "English",
  "preferred_channel": "sms",
  "available_discount": "flat 50%"
}
```


Final Marketing Data:
{
  "ongoing_event": "Ramadan",
  "customer_segment": "Ramadan",
  "marketer_campaign_theme": "New Arrivals",
  "language": "English",
  "preferred_channel": "sms",
  "available_discount": "flat 50%"
}


### with edit option

In [31]:
from typing import Dict, List, TypedDict, Optional
import re
import json

class MarketingState(TypedDict, total=False):
    marketing_data: Dict[str, Optional[str]]
    asked_fields: List[str]

COMMON_TERMS = {
    "customer_segments": ["inactive", "active", "new", "vip", "loyal", "ramadan"],
    "themes": ["new arrivals", "clearance sale", "seasonal collection", "loyalty program", 
               "product launch", "reactivation", "welcome back", "anniversary", "flash sale", 
               "grand opening", "membership", "renewal", "upgrade", "referral", "testimonial",
               "white wednesday", "sale", "generic", "promotional"],
    "channels": ["whatsapp", "email", "sms", "push notification"],
    "offers": ["buy1get1", "30%", "50% off", "free shipping", "discount", "bogo", "flat 50%"],
    "events": ["ramadan", "eid", "christmas", "new year", "black friday", "cyber monday", 
               "valentine's day", "halloween", "back to school", "thanksgiving", 
               "diwali", "lunar new year", "hanukkah", "pride", "easter"],
    "languages": ["english", "arabic", "french", "spanish", "hindi"]
}

def initialize_state() -> MarketingState:
    return {
        "marketing_data": {},
        "asked_fields": []
    }

def extract_offer_details(text: str) -> Dict[str, str]:
    text_lower = text.lower()
    details = {}
    
    # Skip language words in offer detection
    language_terms = COMMON_TERMS["languages"]
    if any(lang in text_lower for lang in language_terms):
        text_lower = re.sub(r'\b(' + '|'.join(language_terms) + r')\b', '', text_lower)
    
    # Special cases first
    if "buy1get1" in text_lower or "bogo" in text_lower:
        details["offer_name"] = "Buy One Get One Free"
    elif "free shipping" in text_lower:
        details["offer_name"] = "Free Shipping"
    else:
        # Percentage discounts
        percent_match = re.search(r"(\d+)%", text_lower)
        if percent_match:
            details["offer_name"] = f"{percent_match.group(1)}% Discount"
        else:
            # Fixed amount discounts
            money_match = re.search(r"(\d+)\s*(rupees|rs|₹)\s*off\s*on\s*(\d+)", text_lower)
            if money_match:
                details["offer_name"] = f"₹{money_match.group(1)} off on ₹{money_match.group(3)}"
            else:
                # Flat discounts
                flat_match = re.search(r"flat\s*(\d+)", text_lower)
                if flat_match:
                    details["offer_name"] = f"Flat {flat_match.group(1)}% Discount"
                else:
                    # Generic offers
                    for offer in COMMON_TERMS["offers"]:
                        if offer in text_lower:
                            details["offer_name"] = offer.title()
                            break
    
    # Offer code
    code_match = re.search(r"(code|coupon)\s*(is)?\s*:?\s*(\w+(?:\s+\w+)*)", text_lower)
    if code_match:
        details["offer_code"] = code_match.group(3).strip().upper()
    elif "coupon" in text_lower or "code" in text_lower:
        details["offer_code"] = None
    
    return details

def extract_info(text: str) -> Dict[str, str]:
    text_lower = text.lower()
    extracted = {}
    
    # Customer segment
    for segment in COMMON_TERMS["customer_segments"]:
        if segment in text_lower:
            extracted["customer_segment"] = segment
            break
    
    # Theme
    for theme in sorted(COMMON_TERMS["themes"], key=len, reverse=True):
        if theme in text_lower:
            extracted["marketer_campaign_theme"] = theme.title()
            break
    
    # Event
    for event in sorted(COMMON_TERMS["events"], key=len, reverse=True):
        if event in text_lower:
            extracted["ongoing_event"] = event.title()
            break
    
    # Channel
    channel_map = {
        "whatsapp": ["whatsapp", "whats app"],
        "email": ["email", "e-mail"],
        "sms": ["sms", "text message"],
        "push notification": ["push", "notification"]
    }
    for channel, variants in channel_map.items():
        if any(variant in text_lower for variant in variants):
            extracted["preferred_channel"] = channel
            break
    
    # Extract offer details
    offer_details = extract_offer_details(text)
    extracted.update(offer_details)
    
    # Language
    for lang in COMMON_TERMS["languages"]:
        if lang in text_lower:
            extracted["language"] = lang.title()
            break
    
    return extracted

def get_next_question(state: MarketingState) -> Optional[str]:
    required_fields = {
        "customer_segment": "Which customer segment should we target? (inactive/active/new/VIP/loyal)",
        "marketer_campaign_theme": "What's the campaign theme? (e.g., New Arrivals, Clearance Sale)",
        "preferred_channel": "Which channel should we use? (WhatsApp/email/SMS)",
        "language": "What language should we use? (English/Arabic/French)"
    }
    
    optional_fields = {
        "ongoing_event": "Is this for any special event? (e.g., Ramadan, Christmas)",
        "offer_name": "What offer would you like to include? (e.g., 30% off, Free Shipping)",
        "offer_code": "Do you have an offer code? (Enter code or 'none')"
    }
    
    # Check required fields first
    for field, question in required_fields.items():
        if field not in state["marketing_data"] and field not in state["asked_fields"]:
            state["asked_fields"].append(field)
            return question
    
    # Then check optional fields
    for field, question in optional_fields.items():
        if field not in state["marketing_data"] and field not in state["asked_fields"]:
            state["asked_fields"].append(field)
            return question
    
    return None

def run_campaign_assistant():
    print("🌟 Marketing Campaign Setup 🌟\n")
    print("You can provide all information in one sentence or answer questions one by one.")
    print("Example: 'Send SMS in English to VIP customers for Ramadan with 50% discount code SUMMER50'\n")
    print("(Type 'quit' anytime to cancel)\n")
    
    state = initialize_state()
    
    # First get initial input
    user_input = input("Tell me about your campaign: ").strip()
    if user_input.lower() == "quit":
        print("Assistant: Goodbye! Campaign setup cancelled.")
        return
    
    # Extract all possible info from first input
    initial_data = extract_info(user_input)
    state["marketing_data"].update(initial_data)
    
    # Get missing information
    while True:
        next_question = get_next_question(state)
        if not next_question:
            break
        
        print(f"\nAssistant: {next_question}")
        
        while True:
            user_input = input("You: ").strip()
            if user_input.lower() == "quit":
                print("Assistant: Goodbye! Campaign setup cancelled.")
                return
            
            if user_input:
                # Handle offer code specifically
                if "offer code?" in next_question.lower():
                    if user_input.lower() != "none":
                        state["marketing_data"]["offer_code"] = user_input.upper()
                    break
                
                # Handle offer name specifically
                if "offer would you like to include?" in next_question.lower():
                    offer_details = extract_offer_details(user_input)
                    if "offer_name" in offer_details:
                        state["marketing_data"]["offer_name"] = offer_details["offer_name"]
                    break
                
                new_data = extract_info(user_input)
                state["marketing_data"].update(new_data)
                break
    
    # Convert to final JSON format
    final_data = {
        "ongoing_event": state["marketing_data"].get("ongoing_event"),
        "customer_segment": state["marketing_data"].get("customer_segment"),
        "marketer_campaign_theme": state["marketing_data"].get("marketer_campaign_theme"),
        "language": state["marketing_data"].get("language"),
        "preferred_channel": state["marketing_data"].get("preferred_channel"),
        "offer_name": state["marketing_data"].get("offer_name"),
        "offer_code": state["marketing_data"].get("offer_code")
    }
    
    # Remove None values
    final_data = {k: v for k, v in final_data.items() if v is not None}
    
    # Display final JSON output
    print("\nFinal Marketing Data:")
    print(json.dumps(final_data, indent=2))
    
    # Enhanced edit option with multi-field support
    while True:
        edit = input("\nWould you like to edit anything? (yes/no): ").strip().lower()
        if edit == "no":
            print("Assistant: Campaign setup complete!")
            break
        elif edit == "yes":
            print("\nCurrent campaign data:")
            fields = list(final_data.keys())
            for i, field in enumerate(fields, 1):
                print(f"{i}. {field}: {final_data.get(field, 'Not set')}")
            
            print("\nEnter field numbers to edit (comma separated, e.g. '1,3,5')")
            print("Or type 'back' to return")
            
            while True:
                selection = input("\nYour selection: ").strip().lower()
                if selection == "back":
                    break
                if selection == "quit":
                    print("Assistant: Goodbye! Campaign setup cancelled.")
                    return
                
                try:
                    selected_indices = [int(idx.strip()) for idx in selection.split(",")]
                    invalid = [idx for idx in selected_indices if not 1 <= idx <= len(fields)]
                    if invalid:
                        print(f"Invalid numbers: {invalid}. Please enter numbers between 1-{len(fields)}")
                        continue
                    
                    # Edit all selected fields
                    updates = {}
                    for idx in selected_indices:
                        field = fields[idx-1]
                        current_value = final_data.get(field, "")
                        new_value = input(f"New value for '{field}' (current: {current_value}): ").strip()
                        
                        # Special handling for different field types
                        if field == "offer_code":
                            new_value = new_value.upper()
                        elif field == "offer_name":
                            offer_details = extract_offer_details(new_value)
                            new_value = offer_details.get("offer_name", new_value)
                        
                        if new_value.lower() != "none":
                            updates[field] = new_value
                        else:
                            updates[field] = None
                    
                    # Apply all updates
                    for field, value in updates.items():
                        final_data[field] = value
                        state["marketing_data"][field] = value
                    
                    print("\nUpdated Marketing Data:")
                    print(json.dumps({k: v for k, v in final_data.items() if v is not None}, indent=2))
                    break
                
                except ValueError:
                    print("Please enter valid numbers separated by commas (e.g. '1,3,5')")
        else:
            print("Please answer with 'yes' or 'no'")

if __name__ == "__main__":
    run_campaign_assistant()

🌟 Marketing Campaign Setup 🌟

You can provide all information in one sentence or answer questions one by one.
Example: 'Send SMS in English to VIP customers for Ramadan with 50% discount code SUMMER50'

(Type 'quit' anytime to cancel)

Tell me about your campaign: I want to send campaign for Inactive customers in ramadan and new arrivals

Assistant: Which channel should we use? (WhatsApp/email/SMS)
You: whatsapp

Assistant: What language should we use? (English/Arabic/French)
You: english

Assistant: What offer would you like to include? (e.g., 30% off, Free Shipping)
You: 5000 off on purchase of 500

Assistant: Do you have an offer code? (Enter code or 'none')
You: get5000

Final Marketing Data:
{
  "ongoing_event": "Ramadan",
  "customer_segment": "inactive",
  "marketer_campaign_theme": "New Arrivals",
  "language": "English",
  "preferred_channel": "whatsapp",
  "offer_code": "GET5000"
}

Would you like to edit anything? (yes/no): no
Assistant: Campaign setup complete!


# without edit option

In [32]:
from typing import Dict, List, TypedDict, Optional
import re
import json

class MarketingState(TypedDict, total=False):
    marketing_data: Dict[str, Optional[str]]
    asked_fields: List[str]

COMMON_TERMS = {
    "customer_segments": ["inactive", "active", "new", "vip", "loyal", "ramadan],
    "themes": ["new arrivals", "clearance sale", "seasonal collection", "loyalty program", 
               "product launch", "reactivation", "welcome back", "anniversary", "flash sale", 
               "grand opening", "membership", "renewal", "upgrade", "referral", "testimonial",
               "white wednesday", "sale", "generic", "promotional"],
    "channels": ["whatsapp", "email", "sms", "push notification"],
    "offers": ["buy1get1", "30%", "50% off", "free shipping", "discount", "bogo", "flat 50%"],
    "events": ["ramadan", "eid", "christmas", "new year", "black friday", "cyber monday", 
               "valentine's day", "halloween", "back to school", "thanksgiving", 
               "diwali", "lunar new year", "hanukkah", "pride", "easter"],
    "languages": ["english", "arabic", "french", "spanish", "hindi"]
}

def initialize_state() -> MarketingState:
    return {
        "marketing_data": {},
        "asked_fields": []
    }

def extract_offer_details(text: str) -> Dict[str, str]:
    text_lower = text.lower()
    details = {}
    
    # Skip language words in offer detection
    language_terms = COMMON_TERMS["languages"]
    if any(lang in text_lower for lang in language_terms):
        text_lower = re.sub(r'\b(' + '|'.join(language_terms) + r')\b', '', text_lower)
    
    # Special cases first
    if "buy1get1" in text_lower or "bogo" in text_lower:
        details["offer_name"] = "Buy One Get One Free"
    elif "free shipping" in text_lower:
        details["offer_name"] = "Free Shipping"
    else:
        # Percentage discounts
        percent_match = re.search(r"(\d+)%", text_lower)
        if percent_match:
            details["offer_name"] = f"{percent_match.group(1)}% Discount"
        else:
            # Fixed amount discounts
            money_match = re.search(r"(\d+)\s*(rupees|rs|₹)\s*off\s*on\s*(\d+)", text_lower)
            if money_match:
                details["offer_name"] = f"₹{money_match.group(1)} off on ₹{money_match.group(3)}"
            else:
                # Flat discounts
                flat_match = re.search(r"flat\s*(\d+)", text_lower)
                if flat_match:
                    details["offer_name"] = f"Flat {flat_match.group(1)}% Discount"
                else:
                    # Generic offers
                    for offer in COMMON_TERMS["offers"]:
                        if offer in text_lower:
                            details["offer_name"] = offer.title()
                            break
    
    # Offer code
    code_match = re.search(r"(code|coupon)\s*(is)?\s*:?\s*(\w+(?:\s+\w+)*)", text_lower)
    if code_match:
        details["offer_code"] = code_match.group(3).strip().upper()
    elif "coupon" in text_lower or "code" in text_lower:
        details["offer_code"] = None
    
    return details

def extract_info(text: str) -> Dict[str, str]:
    text_lower = text.lower()
    extracted = {}
    
    # Customer segment
    for segment in COMMON_TERMS["customer_segments"]:
        if segment in text_lower:
            extracted["customer_segment"] = segment
            break
    
    # Theme
    for theme in sorted(COMMON_TERMS["themes"], key=len, reverse=True):
        if theme in text_lower:
            extracted["marketer_campaign_theme"] = theme.title()
            break
    
    # Event
    for event in sorted(COMMON_TERMS["events"], key=len, reverse=True):
        if event in text_lower:
            extracted["ongoing_event"] = event.title()
            break
    
    # Channel
    channel_map = {
        "whatsapp": ["whatsapp", "whats app"],
        "email": ["email", "e-mail"],
        "sms": ["sms", "text message"],
        "push notification": ["push", "notification"]
    }
    for channel, variants in channel_map.items():
        if any(variant in text_lower for variant in variants):
            extracted["preferred_channel"] = channel
            break
    
    # Extract offer details
    offer_details = extract_offer_details(text)
    extracted.update(offer_details)
    
    # Language
    for lang in COMMON_TERMS["languages"]:
        if lang in text_lower:
            extracted["language"] = lang.title()
            break
    
    return extracted

def get_next_question(state: MarketingState) -> Optional[str]:
    required_fields = {
        "customer_segment": "Which customer segment should we target? (inactive/active/loyal)",
        "marketer_campaign_theme": "What's the campaign theme? (e.g., New Arrivals, Clearance Sale)",
        "preferred_channel": "Which channel should we use? (WhatsApp/email/SMS)",
        "language": "What language should we use? (English/Arabic/French)"
    }
    
    optional_fields = {
        "ongoing_event": "Is this for any special event? (e.g., Ramadan, Christmas)",
        "offer_name": "What offer would you like to include? (e.g., 30% off, Free Shipping)",
        "offer_code": "Do you have an offer code? (Enter code or 'none')"
    }
    
    # Check required fields first
    for field, question in required_fields.items():
        if field not in state["marketing_data"] and field not in state["asked_fields"]:
            state["asked_fields"].append(field)
            return question
    
    # Then check optional fields
    for field, question in optional_fields.items():
        if field not in state["marketing_data"] and field not in state["asked_fields"]:
            state["asked_fields"].append(field)
            return question
    
    return None

def run_campaign_assistant():
    print("🌟 Marketing Campaign Setup 🌟\n")
    print("You can provide all information in one sentence or answer questions one by one.")
    print("Example: 'Send SMS in English to active customers for Ramadan with 50% discount code SUMMER50'\n")
    print("(Type 'quit' anytime to cancel)\n")
    
    state = initialize_state()
    
    # First get initial input
    user_input = input("Tell me about your campaign: ").strip()
    if user_input.lower() == "quit":
        print("Assistant: Goodbye! Campaign setup cancelled.")
        return
    
    # Extract all possible info from first input
    initial_data = extract_info(user_input)
    state["marketing_data"].update(initial_data)
    
    # Get missing information
    while True:
        next_question = get_next_question(state)
        if not next_question:
            break
        
        print(f"\nAssistant: {next_question}")
        
        while True:
            user_input = input("You: ").strip()
            if user_input.lower() == "quit":
                print("Assistant: Goodbye! Campaign setup cancelled.")
                return
            
            if user_input:
                # Handle offer code specifically
                if "offer code?" in next_question.lower():
                    if user_input.lower() != "none":
                        state["marketing_data"]["offer_code"] = user_input.upper()
                    break
                
                # Handle offer name specifically
                if "offer would you like to include?" in next_question.lower():
                    offer_details = extract_offer_details(user_input)
                    if "offer_name" in offer_details:
                        state["marketing_data"]["offer_name"] = offer_details["offer_name"]
                    break
                
                new_data = extract_info(user_input)
                state["marketing_data"].update(new_data)
                break
    
    # Convert to final JSON format
    final_data = {
        "ongoing_event": state["marketing_data"].get("ongoing_event"),
        "customer_segment": state["marketing_data"].get("customer_segment"),
        "marketer_campaign_theme": state["marketing_data"].get("marketer_campaign_theme"),
        "language": state["marketing_data"].get("language"),
        "preferred_channel": state["marketing_data"].get("preferred_channel"),
        "offer_name": state["marketing_data"].get("offer_name"),
        "offer_code": state["marketing_data"].get("offer_code")
    }
    
    # Remove None values
    final_data = {k: v for k, v in final_data.items() if v is not None}
    
    # Display final JSON output
    print("\nFinal Marketing Data:")
    print(json.dumps(final_data, indent=2))
    print("\nAssistant: Campaign setup complete!")

if __name__ == "__main__":
    run_campaign_assistant()

🌟 Marketing Campaign Setup 🌟

You can provide all information in one sentence or answer questions one by one.
Example: 'Send SMS in English to VIP customers for Ramadan with 50% discount code SUMMER50'

(Type 'quit' anytime to cancel)

Tell me about your campaign: I want to send campaign for Inactive customers in ramadan and new arrivals and language english on whatsapp and buy1get1 and use code get1

Final Marketing Data:
{
  "ongoing_event": "Ramadan",
  "customer_segment": "inactive",
  "marketer_campaign_theme": "New Arrivals",
  "language": "English",
  "preferred_channel": "whatsapp",
  "offer_name": "Buy One Get One Free",
  "offer_code": "GET1"
}

Assistant: Campaign setup complete!


# Final1

In [75]:
from typing import Dict, List, TypedDict, Optional
import re
import json
import pandas as pd

# -----------------------------------------------------------------------------
# Define the conversation state for campaign setup.
# -----------------------------------------------------------------------------
class MarketingState(TypedDict, total=False):
    marketing_data: Dict[str, Optional[str]]
    asked_fields: List[str]

# -----------------------------------------------------------------------------
# Define common terms for extraction.
# -----------------------------------------------------------------------------
COMMON_TERMS = {
    "customer_segments": ["inactive", "active", "new", "repeat","Enrolled","Acquired_cust"],
    "themes": ["new arrivals", "clearance sale", "seasonal collection", "loyalty program", 
               "product launch", "reactivation", "welcome back", "anniversary", "flash sale", 
               "grand opening", "membership", "renewal", "upgrade", "referral", "testimonial",
               "white wednesday", "sale", "generic", "promotional"],
    "channels": ["whatsapp", "email", "sms"],
    "offers": ["buy1get1", "30%", "50% off", "free shipping", "discount", "bogo", "flat 50%"],
    "events": ["ramadan", "eid", "christmas", "new year", "black friday", "cyber monday", 
               "valentine's day", "halloween", "back to school", "thanksgiving", 
               "diwali", "lunar new year", "hanukkah", "pride", "easter"],
    "languages": ["english", "arabic"]
}

# -----------------------------------------------------------------------------
# Initialize conversation state.
# -----------------------------------------------------------------------------
def initialize_state() -> MarketingState:
    return {
        "marketing_data": {},
        "asked_fields": []
    }

# -----------------------------------------------------------------------------
# Extract offer details from text.
# -----------------------------------------------------------------------------
def extract_offer_details(text: str) -> Dict[str, str]:
    text_lower = text.lower()
    details = {}
    
    # Remove language words to avoid interference.
    language_terms = COMMON_TERMS["languages"]
    if any(lang in text_lower for lang in language_terms):
        text_lower = re.sub(r'\b(' + '|'.join(language_terms) + r')\b', '', text_lower)
    
    # Special cases first.
    if "buy1get1" in text_lower or "bogo" in text_lower:
        details["offer_name"] = "Buy One Get One Free"
    elif "free shipping" in text_lower:
        details["offer_name"] = "Free Shipping"
    else:
        # Percentage discounts.
        percent_match = re.search(r"(\d+)%", text_lower)
        if percent_match:
            details["offer_name"] = f"{percent_match.group(1)}% Discount"
        else:
            # Fixed amount discounts.
            money_match = re.search(r"(\d+)\s*(rupees|rs|₹)\s*off\s*on\s*(\d+)", text_lower)
            if money_match:
                details["offer_name"] = f"₹{money_match.group(1)} off on ₹{money_match.group(3)}"
            else:
                # Flat discounts.
                flat_match = re.search(r"flat\s*(\d+)", text_lower)
                if flat_match:
                    details["offer_name"] = f"Flat {flat_match.group(1)}% Discount"
                else:
                    # Generic offers.
                    for offer in COMMON_TERMS["offers"]:
                        if offer in text_lower:
                            details["offer_name"] = offer.title()
                            break
    
    # Offer code extraction.
    code_match = re.search(r"(code|coupon)\s*(is)?\s*:?\s*(\w+(?:\s+\w+)*)", text_lower)
    if code_match:
        details["offer_code"] = code_match.group(3).strip().upper()
    elif "coupon" in text_lower or "code" in text_lower:
        details["offer_code"] = None
    
    return details

# -----------------------------------------------------------------------------
# Extract general campaign info from text.
# -----------------------------------------------------------------------------
def extract_info(text: str) -> Dict[str, str]:
    text_lower = text.lower()
    extracted = {}
    
    # Customer segment extraction.
    for segment in COMMON_TERMS["customer_segments"]:
        if segment in text_lower:
            extracted["customer_segment"] = segment
            break
    
    # Theme extraction.
    for theme in sorted(COMMON_TERMS["themes"], key=len, reverse=True):
        if theme in text_lower:
            extracted["marketer_campaign_theme"] = theme.title()
            break
    
    # Event extraction.
    for event in sorted(COMMON_TERMS["events"], key=len, reverse=True):
        if event in text_lower:
            extracted["ongoing_event"] = event.title()
            break
    
    # Channel extraction.
    channel_map = {
        "whatsapp": ["whatsapp", "whats app"],
        "email": ["email", "e-mail"],
        "sms": ["sms", "text message"]
    }
    for channel, variants in channel_map.items():
        if any(variant in text_lower for variant in variants):
            extracted["preferred_channel"] = channel
            break
    
    # Extract offer details.
    offer_details = extract_offer_details(text)
    extracted.update(offer_details)
    
    # Language extraction.
    for lang in COMMON_TERMS["languages"]:
        if lang in text_lower:
            extracted["language"] = lang.title()
            break
    
    return extracted

# -----------------------------------------------------------------------------
# Determine the next question to ask.
# -----------------------------------------------------------------------------
def get_next_question(state: MarketingState) -> Optional[str]:
    required_fields = {
        "customer_segment": "Which customer segment should we target? (e.g., inactive, active, loyal, new)",
        "marketer_campaign_theme": "What's the campaign theme? (e.g., New Arrivals, Clearance Sale)",
        "preferred_channel": "Which channel should we use? (e.g., WhatsApp, email, SMS)",
        "language": "What language should we use? (e.g., English, Arabic, French)"
    }
    
    optional_fields = {
        "ongoing_event": "Is this for a special event? (e.g., Ramadan, Christmas)",
        "offer_name": "What offer would you like to include? (e.g., 30% off, Free Shipping)",
        "offer_code": "Do you have an offer code? (Enter code or 'none')"
    }
    
    # Check required fields first.
    for field, question in required_fields.items():
        if field not in state["marketing_data"] and field not in state["asked_fields"]:
            state["asked_fields"].append(field)
            return question
    
    # Then check optional fields.
    for field, question in optional_fields.items():
        if field not in state["marketing_data"] and field not in state["asked_fields"]:
            state["asked_fields"].append(field)
            return question
    
    return None

# -----------------------------------------------------------------------------
# Sample campaign data (as provided)
# -----------------------------------------------------------------------------
SAMPLE_DATA = [
    {"Campaign name": "Ramadan", "Customer Segment": "New Customer", "Time": "Friday-Evening", "Offer": "20%", "Response Rates": "4%"},
    {"Campaign name": "Ramadan", "Customer Segment": "Winback customer", "Time": "Friday-Evening", "Offer": "Buy1Get1", "Response Rates": "0.50%"},
    {"Campaign name": "Ramadan", "Customer Segment": "Active Customer", "Time": "Friday-Evening", "Offer": "10% off", "Response Rates": "3%"},
    {"Campaign name": "Ramadan", "Customer Segment": "Repeat Customer", "Time": "Friday-Evening", "Offer": "30% off", "Response Rates": "3%"},
    {"Campaign name": "O2O", "Customer Segment": "New Customer", "Time": "Sunday morning", "Offer": "20% Off", "Response Rates": "4%"},
    {"Campaign name": "O2O", "Customer Segment": "Winback customer", "Time": "Sunday morning", "Offer": "FLAT 60% off", "Response Rates": "0.50%"},
    {"Campaign name": "O2O", "Customer Segment": "Active Customer", "Time": "Sunday morning", "Offer": "No offer", "Response Rates": "3%"},
    {"Campaign name": "O2O", "Customer Segment": "Repeat Customer", "Time": "Sunday morning", "Offer": "10% discount", "Response Rates": "3%"}
]

# -----------------------------------------------------------------------------
# Main function: decide intent and either aggregate campaign plan or run interactive flow.
# -----------------------------------------------------------------------------
def run_campaign_assistant():
    print("🌟 Marketing Campaign Setup 🌟\n")
    print("You can either ask for a full campaign plan (e.g., 'Please plan a Ramadan campaign for me' or 'please make a campaign for o2o') or provide details step by step.")
    print("(Type 'quit' anytime to cancel)\n")
    
    state = initialize_state()
    
    user_input_text = input("Tell me about your campaign: ").strip()
    if user_input_text.lower() == "quit":
        print("Assistant: Goodbye! Campaign setup cancelled.")
        return
    
    # --- Intent Detection ---
    # Updated keywords now include 'plan', 'build', 'optimise', 'make', 'create'
    campaign_names = ["ramadan", "o2o"]
    intent_keywords = ["plan", "build", "optimise", "make", "create"]
    if any(camp in user_input_text.lower() for camp in campaign_names) and any(kw in user_input_text.lower() for kw in intent_keywords):
        # Determine the campaign name.
        campaign_name = None
        for name in campaign_names:
            if name in user_input_text.lower():
                campaign_name = name.capitalize()
                break
        # Filter the sample data.
        campaign_data = [row for row in SAMPLE_DATA if row["Campaign name"].lower() == campaign_name.lower()]
        if campaign_data:
            print(f"\nAssistant: Pulling the data for the entire {campaign_name} campaign!\n")
            df = pd.DataFrame(campaign_data)
            print(df.to_string(index=False))
            # Suggest the best customer segment (maximizing response rates).
            try:
                best_segment = max(campaign_data, key=lambda x: float(x["Response Rates"].replace("%", "")))
                print("\nAssistant: Based on the data, the best customer segment to target is '{}' at '{}' with the offer '{}' to maximize response rates.".format(
                    best_segment["Customer Segment"],
                    best_segment["Time"],
                    best_segment["Offer"]
                ))
            except Exception as e:
                print("Assistant: There was an error processing the data. Please try again later.")
            return
        else:
            print("Assistant: Sorry, no data is available for that campaign.")
            return
    
    # --- Interactive Flow ---
    # If not an aggregate campaign request, run the interactive detail collection.
    initial_data = extract_info(user_input_text)
    state["marketing_data"].update(initial_data)
    
    while True:
        next_question = get_next_question(state)
        if not next_question:
            break
        
        print(f"\nAssistant: {next_question}")
        user_input_text = input("You: ").strip()
        if user_input_text.lower() == "quit":
            print("Assistant: Goodbye! Campaign setup cancelled.")
            return
        
        # Special handling for offer code.
        if "offer code" in next_question.lower():
            if user_input_text.lower() != "none":
                state["marketing_data"]["offer_code"] = user_input_text.upper()
        # Special handling for offer name.
        elif "offer would you like to include" in next_question.lower():
            offer_details = extract_offer_details(user_input_text)
            if "offer_name" in offer_details:
                state["marketing_data"]["offer_name"] = offer_details["offer_name"]
        else:
            new_data = extract_info(user_input_text)
            state["marketing_data"].update(new_data)
    
    final_data = {
        "ongoing_event": state["marketing_data"].get("ongoing_event"),
        "customer_segment": state["marketing_data"].get("customer_segment"),
        "marketer_campaign_theme": state["marketing_data"].get("marketer_campaign_theme"),
        "language": state["marketing_data"].get("language"),
        "preferred_channel": state["marketing_data"].get("preferred_channel"),
        "offer_name": state["marketing_data"].get("offer_name"),
        "offer_code": state["marketing_data"].get("offer_code")
    }
    final_data = {k: v for k, v in final_data.items() if v is not None}
    
    print("\nFinal Marketing Data:")
    print(json.dumps(final_data, indent=2))
    print("\nAssistant: Campaign setup complete!")

if __name__ == "__main__":
    run_campaign_assistant()

🌟 Marketing Campaign Setup 🌟

You can either ask for a full campaign plan (e.g., 'Please plan a Ramadan campaign for me' or 'please make a campaign for o2o') or provide details step by step.
(Type 'quit' anytime to cancel)

Tell me about your campaign: Create a campaign for Diwali with Active base and English Language

Assistant: What's the campaign theme? (e.g., New Arrivals, Clearance Sale)
You: New Arriavals

Assistant: Which channel should we use? (e.g., WhatsApp, email, SMS)
You: ALL channels

Assistant: What offer would you like to include? (e.g., 30% off, Free Shipping)
You: 10% off

Assistant: Do you have an offer code? (Enter code or 'none')
You: Nono

Final Marketing Data:
{
  "ongoing_event": "Diwali",
  "customer_segment": "new",
  "language": "English",
  "offer_name": "10% Discount",
  "offer_code": "NONO"
}

Assistant: Campaign setup complete!


In [74]:
from typing import Dict, TypedDict, Optional
from openai import AzureOpenAI
import json

class MessageState(TypedDict):
    campaign_data: Dict
    generated_message: Optional[str]
    error: Optional[str]

class CampaignMessageGenerator:
    def __init__(self, azure_endpoint: str, api_key: str, api_version: str):
        self.client = AzureOpenAI(
            azure_endpoint=azure_endpoint,
            api_key=api_key,
            api_version=api_version
        )
        
        # Define channel-specific configurations
        self.channel_config = {
            "sms": {
                "max_chars": 160,
                "emoji_suggestion": "Use minimal emojis",
                "template": self._base_template()
            },
            "whatsapp": {
                "max_chars": 250,
                "emoji_suggestion": "You can use emojis",
                "template": self._whatsapp_template()
            },
            "email": {
                "max_chars": 500,
                "emoji_suggestion": "Use emojis sparingly",
                "template": self._email_template()
            }
        }
    
    def _base_template(self) -> str:
        return """Create ONE marketing message based on these specifics:

CUSTOMER PROFILE:
- Segment: {customer_segment}
{preferred_channel_line}

CAMPAIGN DETAILS:
{event_line}
{theme_line}
{offer_lines}

TECHNICAL REQUIREMENTS:
- MUST be under {max_chars} characters
- Channel: {preferred_channel}
- Language: {language}
{emoji_line}
{personalization_line}

Respond with JUST the message text, no explanations.
"""

    def _whatsapp_template(self) -> str:
        return self._base_template() + "\n- Make it conversational and friendly\n- Can use WhatsApp formatting (emojis, *bold*, _italic_)"

    def _email_template(self) -> str:
        return self._base_template() + "\n- Include a clear call-to-action\n- More formal than WhatsApp"

    def _validate_input(self, data: Dict) -> Optional[str]:
        required_fields = [
            'customer_segment', 
            'preferred_channel',
            'language'
        ]
        
        for field in required_fields:
            if field not in data:
                return f"Missing required field: {field}"
            
        if data['preferred_channel'] not in self.channel_config:
            return f"Invalid channel. Supported: {', '.join(self.channel_config.keys())}"
        
        return None

    def _build_prompt(self, data: Dict) -> str:
        channel = data['preferred_channel'].lower()
        config = self.channel_config[channel]
        
        # Prepare dynamic parts of the template
        template_data = {
            'max_chars': config['max_chars'],
            'emoji_line': config['emoji_suggestion'],
            'preferred_channel_line': f"- Preferred channel: {channel}",
            'language': data.get('language', 'English'),
            'customer_segment': data['customer_segment'],
            'preferred_channel': channel
        }
        
        # Handle optional fields
        template_data['event_line'] = f"- Ongoing event: {data['ongoing_event']}\n" if 'ongoing_event' in data else ""
        template_data['theme_line'] = f"- Campaign theme: {data['marketer_campaign_theme']}\n" if 'marketer_campaign_theme' in data else ""
        
        # Handle offers
        offer_parts = []
        if 'offer_name' in data:
            offer_parts.append(f"- Offer: {data['offer_name']}")
        if 'offer_code' in data:
            offer_parts.append(f"- Offer code: {data['offer_code']}")
        if 'available_discount' in data:
            offer_parts.append(f"- Discount: {data['available_discount']}")
            
        template_data['offer_lines'] = '\n'.join(offer_parts) if offer_parts else "- No specific offers"
        
        # Personalization
        if data.get('personalization_allowed', False):
            template_data['personalization_line'] = "- CAN include personalization if appropriate"
        else:
            template_data['personalization_line'] = "- DO NOT include personalization"
        
        # Get the appropriate template
        template = config['template']
        
        return template.format(**template_data)

    def generate_message(self, campaign_data: Dict) -> MessageState:
        # Validate input
        error = self._validate_input(campaign_data)
        if error:
            return {
                "campaign_data": campaign_data,
                "generated_message": None,
                "error": error
            }
        
        try:
            prompt = self._build_prompt(campaign_data)
            
            response = self.client.chat.completions.create(
                model="gpt-4",
                messages=[{"role": "user", "content": prompt}],
                max_tokens=self.channel_config[campaign_data['preferred_channel']]['max_chars'],
                temperature=0.7
            )
            
            message = response.choices[0].message.content.strip()
            
            return {
                "campaign_data": campaign_data,
                "generated_message": message,
                "error": None
            }
            
        except Exception as e:
            return {
                "campaign_data": campaign_data,
                "generated_message": None,
                "error": f"Generation failed: {str(e)}"
            }

def load_campaign_data(json_input: str) -> Dict:
    try:
        return json.loads(json_input)
    except json.JSONDecodeError:
        return {"error": "Invalid JSON input"}

def main():
    # Azure OpenAI configuration
    azure_config = {
        "azure_endpoint": "https://lmgdllaoidev2.openai.azure.com/",
        "api_key": "b02e94fb7b024b63a18a0161de281416",
        "api_version": "2025-01-01-preview"
    }
    
    # Initialize generator
    generator = CampaignMessageGenerator(**azure_config)
    
    # Example JSON input (can be replaced with actual JSON input)
    campaign_json = """{
  "ongoing_event": "Diwali",
  "customer_segment": "new",
  "marketer_campaign_theme": "New Arrivals",
  "language": "English",
  "preferred_channel": "sms",
  "offer_name": "Free Shipping",
  "offer_code": "DIWALI100"
}
"""
    {
  "ongoing_event": "Diwali",
  "customer_segment": "new",
  "language": "English",
  "offer_name": "10% Discount",
  "offer_code": "NONO"
}
    
    # Load and validate JSON
    campaign_data = load_campaign_data(campaign_json)
    if "error" in campaign_data:
        print(f"Error: {campaign_data['error']}")
        return
    
    # Generate message
    result = generator.generate_message(campaign_data)
    
    # Display results
    print("\nCampaign Data:")
    print(json.dumps(result['campaign_data'], indent=2))
    
    if result['error']:
        print(f"\nError: {result['error']}")
    else:
        print("\nGenerated Message:")
        print(result['generated_message'])
        
        # Verify character count
        channel = result['campaign_data']['preferred_channel']
        max_chars = generator.channel_config[channel]['max_chars']
        char_count = len(result['generated_message'])
        print(f"\nCharacter count: {char_count}/{max_chars}")
        if char_count > max_chars:
            print("⚠️ Warning: Message exceeds recommended character limit!")

if __name__ == "__main__":
    main()


Campaign Data:
{
  "ongoing_event": "Diwali",
  "customer_segment": "new",
  "marketer_campaign_theme": "New Arrivals",
  "language": "English",
  "preferred_channel": "sms",
  "offer_name": "Free Shipping",
  "offer_code": "DIWALI100"
}

Generated Message:
Celebrate Diwali with our New Arrivals! Enjoy Free Shipping on your order with code DIWALI100. Shop now and light up your celebration! 🎉

Character count: 136/160


In [85]:
from typing import Dict, List, TypedDict, Optional
import re
import json
import pandas as pd
from openai import AzureOpenAI

# Azure OpenAI configuration
azure_config = {
    "azure_endpoint": "https://lmgdllaoidev2.openai.azure.com/",
    "api_key": "b02e94fb7b024b63a18a0161de281416",
    "api_version": "2025-01-01-preview"
}

# Initialize Azure OpenAI client
client = AzureOpenAI(
    azure_endpoint=azure_config["azure_endpoint"],
    api_key=azure_config["api_key"],
    api_version=azure_config["api_version"]
)

# -----------------------------------------------------------------------------
# Define the conversation state for campaign setup.
# -----------------------------------------------------------------------------
class MarketingState(TypedDict, total=False):
    marketing_data: Dict[str, Optional[str]]
    asked_fields: List[str]
    conversation_history: List[Dict[str, str]]

# -----------------------------------------------------------------------------
# Initialize conversation state.
# -----------------------------------------------------------------------------
def initialize_state() -> MarketingState:
    return {
        "marketing_data": {},
        "asked_fields": [],
        "conversation_history": []
    }

# -----------------------------------------------------------------------------
# Generate conversational response using OpenAI
# -----------------------------------------------------------------------------
def generate_ai_response(prompt: str, conversation_history: List[Dict[str, str]]) -> str:
    messages = conversation_history.copy()
    messages.append({"role": "user", "content": prompt})
    
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=messages,
            temperature=0.7,
            max_tokens=150
        )
        return response.choices[0].message.content
    except Exception as e:
        print(f"Error calling OpenAI: {e}")
        return "I'm having trouble processing that. Could you please rephrase or provide more details?"

# -----------------------------------------------------------------------------
# Extract structured data from natural language using OpenAI
# -----------------------------------------------------------------------------
def extract_info_with_ai(text: str, fields_to_extract: List[str]) -> Dict[str, str]:
    prompt = f"""Extract the following marketing campaign details from this text:
    {text}
    
    Look for these specific fields: {', '.join(fields_to_extract)}.
    Return ONLY a JSON dictionary with the extracted values or null if not found.
    Example: {{"customer_segment": "new", "marketer_campaign_theme": "Ramadan"}}
    """
    
    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3,
            max_tokens=200,
            response_format={"type": "json_object"}
        )
        extracted_data = json.loads(response.choices[0].message.content)
        return {k: v for k, v in extracted_data.items() if v is not None}
    except Exception as e:
        print(f"Error extracting info with AI: {e}")
        return {}

# -----------------------------------------------------------------------------
# Sample campaign data
# -----------------------------------------------------------------------------
SAMPLE_DATA = [
    {"Campaign name": "Ramadan", "Customer Segment": "New Customer", "Time": "Friday-Evening", "Offer": "20%", "Response Rates": "4%"},
    {"Campaign name": "Ramadan", "Customer Segment": "Winback customer", "Time": "Friday-Evening", "Offer": "Buy1Get1", "Response Rates": "0.50%"},
    {"Campaign name": "Ramadan", "Customer Segment": "Active Customer", "Time": "Friday-Evening", "Offer": "10% off", "Response Rates": "3%"},
    {"Campaign name": "Ramadan", "Customer Segment": "Repeat Customer", "Time": "Friday-Evening", "Offer": "30% off", "Response Rates": "3%"},
    {"Campaign name": "O2O", "Customer Segment": "New Customer", "Time": "Sunday morning", "Offer": "20% Off", "Response Rates": "4%"},
    {"Campaign name": "O2O", "Customer Segment": "Winback customer", "Time": "Sunday morning", "Offer": "FLAT 60% off", "Response Rates": "0.50%"},
    {"Campaign name": "O2O", "Customer Segment": "Active Customer", "Time": "Sunday morning", "Offer": "No offer", "Response Rates": "3%"},
    {"Campaign name": "O2O", "Customer Segment": "Repeat Customer", "Time": "Sunday morning", "Offer": "10% discount", "Response Rates": "3%"}
]

# -----------------------------------------------------------------------------
# Get the next question based on missing fields
# -----------------------------------------------------------------------------
def get_next_question(state: MarketingState) -> Optional[str]:
    required_fields = {
        "customer_segment": "Who is your target audience? (new customers, loyal customers, etc.)",
        "marketer_campaign_theme": "What's the main theme or purpose of your campaign?",
        "preferred_channel": "Which communication channel will you use? (email, SMS, WhatsApp)",
        "language": "What language should the campaign use?"
    }
    
    optional_fields = {
        "ongoing_event": "Is this campaign tied to a specific event or holiday?",
        "offer_name": "What special offer are you promoting, if any?",
        "offer_code": "Do you have a specific promo code to include?"
    }
    
    # Check required fields first
    for field, question in required_fields.items():
        if field not in state["marketing_data"] and field not in state["asked_fields"]:
            state["asked_fields"].append(field)
            return question
    
    # Then check optional fields
    for field, question in optional_fields.items():
        if field not in state["marketing_data"] and field not in state["asked_fields"]:
            state["asked_fields"].append(field)
            return question
    
    return None

# -----------------------------------------------------------------------------
# Main function: run the campaign assistant
# -----------------------------------------------------------------------------
def run_campaign_assistant():
    print("🌟 Welcome to the Marketing Campaign Assistant! 🌟\n")
    print("I can help you plan a new marketing campaign or optimize an existing one.")
    print("You can describe your campaign in detail or answer questions one by one.")
    print("(Type 'quit' anytime to exit)\n")
    
    state = initialize_state()
    state["conversation_history"].append({
        "role": "system",
        "content": "You are a helpful marketing campaign assistant. Guide the user through setting up a campaign with a friendly, professional tone."
    })
    
    # Initial user input
    user_input = input("You: ").strip()
    if user_input.lower() == "quit":
        print("\nAssistant: Goodbye! Let me know if you need help with campaigns later.")
        return
    
    # Check if user wants a pre-defined campaign
    campaign_names = ["ramadan", "o2o"]
    if any(name in user_input.lower() for name in campaign_names):
        campaign_name = next((name for name in campaign_names if name in user_input.lower()), None)
        campaign_data = [row for row in SAMPLE_DATA if row["Campaign name"].lower() == campaign_name.lower()]
        
        if campaign_data:
            print(f"\nAssistant: Here's the data for the {campaign_name.capitalize()} campaign:\n")
            df = pd.DataFrame(campaign_data)
            print(df.to_string(index=False))
            
            try:
                best_segment = max(campaign_data, key=lambda x: float(x["Response Rates"].replace("%", "")))
                ai_response = generate_ai_response(
                    f"Based on this campaign data: {campaign_data}, suggest the best approach.",
                    state["conversation_history"]
                )
                print(f"\nAssistant: {ai_response}")
            except Exception as e:
                print("\nAssistant: Based on the data, the most effective segment appears to be " +
                      f"'{best_segment['Customer Segment']}' with a {best_segment['Response Rates']} response rate.")
            return
    
    # Extract initial information using AI
    fields_to_extract = [
        "customer_segment", "marketer_campaign_theme", 
        "preferred_channel", "language", "ongoing_event",
        "offer_name", "offer_code"
    ]
    extracted_data = extract_info_with_ai(user_input, fields_to_extract)
    state["marketing_data"].update(extracted_data)
    state["conversation_history"].append({"role": "user", "content": user_input})
    
    # Interactive question flow
    while True:
        next_question = get_next_question(state)
        if not next_question:
            break
        
        # Generate a friendly version of the question using AI
        friendly_question = generate_ai_response(
            f"Ask this marketing campaign question in a friendly, conversational way: {next_question}",
            state["conversation_history"]
        )
        
        print(f"\nAssistant: {friendly_question}")
        user_input = input("You: ").strip()
        
        if user_input.lower() == "quit":
            print("\nAssistant: Campaign setup cancelled. Let me know if you need help later!")
            return
        
        # Extract information from the response
        extracted_data = extract_info_with_ai(user_input, [state["asked_fields"][-1]])
        state["marketing_data"].update(extracted_data)
        state["conversation_history"].append({"role": "user", "content": user_input})
    
    # Prepare final output
    final_data = {
        "ongoing_event": state["marketing_data"].get("ongoing_event"),
        "customer_segment": state["marketing_data"].get("customer_segment"),
        "marketer_campaign_theme": state["marketing_data"].get("marketer_campaign_theme"),
        "language": state["marketing_data"].get("language"),
        "preferred_channel": state["marketing_data"].get("preferred_channel"),
        "offer_name": state["marketing_data"].get("offer_name"),
        "offer_code": state["marketing_data"].get("offer_code")
    }
    final_data = {k: v for k, v in final_data.items() if v is not None}
    
    # Generate a summary using AI
    summary_prompt = f"""Create a concise summary of this marketing campaign plan:
    {json.dumps(final_data, indent=2)}
    
    Include a friendly conclusion and remind the user they can make changes if needed.
    """
    ai_summary = generate_ai_response(summary_prompt, state["conversation_history"])
    
    print("\n🌟 Campaign Summary 🌟")
    print(ai_summary)
    print("\nHere's your complete campaign plan in JSON format:")
    print(json.dumps(final_data, indent=2))
    print("\nAssistant: Your campaign is ready to go! Would you like to make any adjustments?")

if __name__ == "__main__":
    run_campaign_assistant()

🌟 Welcome to the Marketing Campaign Assistant! 🌟

I can help you plan a new marketing campaign or optimize an existing one.
You can describe your campaign in detail or answer questions one by one.
(Type 'quit' anytime to exit)

You: i want send campaign for inactive customer

Assistant: That sounds like a fantastic idea! To better assist you, could you share more about the main theme or purpose of the campaign you're envisioning for your inactive customers? What specific message or feeling would you like to convey to re-engage them?
You: i want campaign for ramadan

Assistant: That sounds like a fantastic idea for a campaign! Just wondering, which communication channel were you thinking of using for this? Are you leaning towards email, SMS, or perhaps WhatsApp?
You: sms

Assistant: That sounds like a great initiative! Just to make sure we connect well with your audience, what language would you like to use for this SMS campaign?
You: english

Assistant: Great choice for a campaign! Jus